In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = 'E:\huggingface_cache'

In [2]:
from unstructured.partition.auto import partition


In [2]:
from pathlib import Path

# Unstructured

In [3]:
base_path = Path('c:/Users/BaukeBrenninkmeijer/Downloads/')
files = base_path.glob('**/*')
list(files)

[WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/00000-3634596539.png'),
 WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/00100srPORTRAIT_00100_BURST20210709223554795_COVER.jpg'),
 WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/00100srPORTRAIT_00100_BURST20210809152912941_COVER.jpg'),
 WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/00100srPORTRAIT_00100_BURST20210904215334697_COVER.jpg'),
 WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/00100srPORTRAIT_00100_BURST20211021230056673_COVER.jpg'),
 WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/00100srPORTRAIT_00100_BURST20211030122001250_COVER.jpg'),
 WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/111002.jpg'),
 WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/111003.jpg'),
 WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/11101779123.zip'),
 WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/1450864397-liamhemsworthcosmopolitanuk.jpg'),
 WindowsPath('c:/Users/BaukeBrenninkmeijer/Downloads/1589343_0.

In [8]:
elements = []
for f in list(base_path.glob('**/*'))[:10]:
    print(f'Ingesting {str(f)}')
    try:
        elements += partition(filename=f.as_posix())
    except ValueError as e:
        print(f'Couldnt ingest {str(f)} due to {e}')

Ingesting c:\Users\BaukeBrenninkmeijer\Downloads\00000-3634596539.png
Ingesting c:\Users\BaukeBrenninkmeijer\Downloads\00100srPORTRAIT_00100_BURST20210709223554795_COVER.jpg
Ingesting c:\Users\BaukeBrenninkmeijer\Downloads\00100srPORTRAIT_00100_BURST20210809152912941_COVER.jpg
Ingesting c:\Users\BaukeBrenninkmeijer\Downloads\00100srPORTRAIT_00100_BURST20210904215334697_COVER.jpg
Ingesting c:\Users\BaukeBrenninkmeijer\Downloads\00100srPORTRAIT_00100_BURST20211021230056673_COVER.jpg
Ingesting c:\Users\BaukeBrenninkmeijer\Downloads\00100srPORTRAIT_00100_BURST20211030122001250_COVER.jpg
Ingesting c:\Users\BaukeBrenninkmeijer\Downloads\111002.jpg
Ingesting c:\Users\BaukeBrenninkmeijer\Downloads\111003.jpg
Ingesting c:\Users\BaukeBrenninkmeijer\Downloads\11101779123.zip
Couldnt ingest c:\Users\BaukeBrenninkmeijer\Downloads\11101779123.zip due to Invalid file c:/Users/BaukeBrenninkmeijer/Downloads/11101779123.zip. The FileType.ZIP file type is not supported in partition.
Ingesting c:\Users\Ba

# Loading data into llama-index

In [2]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader

In [3]:
documents = SimpleDirectoryReader('c:/Users/BaukeBrenninkmeijer/Downloads/', num_files_limit=10).load_data()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


# Configure HuggingFace model

In [15]:
del service_context
del llm

In [22]:
from llama_index.prompts import PromptTemplate
from llama_index import ServiceContext

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

import torch
from llama_index.llms import HuggingFaceLLM
finished = False
llm = HuggingFaceLLM(
    context_window=4096, 
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="StabilityAI/stablelm-tuned-alpha-3b",
    model_name="StabilityAI/stablelm-tuned-alpha-3b",
    device_map="cuda",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16, 'offload_folder': '/offloaded_weights'},
)
service_context = ServiceContext.from_defaults(
    chunk_size=1024, 
    llm=llm,
    embed_model="local"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024, 
    llm=llm,
    embed_model="local"
)

In [24]:
llm

'cuda'

In [26]:
llm.complete('test').text

c:\Users\BaukeBrenninkmeijer\miniconda3\envs\p310gpu\lib\site-packages\transformers\generation\configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


"Hello! How can I help you today? Do you have a specific question or task you'd like me to assist you with?"

In [10]:
service_context.embed_model

LangchainEmbedding(model_name='BAAI/bge-small-en', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x0000012B206BE3B0>)

In [11]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [12]:
index.storage_context.persist("document_index")

In [27]:
from llama_index import StorageContext, load_index_from_storage

# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="document_index")

# Load index from the storage context
new_index = load_index_from_storage(storage_context)

In [28]:
chat_engine = index.as_chat_engine()
chat_engine

In [29]:
chat_engine.chat('Breaky breaky, do you copy?')

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


AgentChatResponse(response='user: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky breaky, do you copy? \nassistant: \nuser: Breaky', sources=[], source_nodes=[])

In [30]:
# chat_engine = index.as_chat_engine()
streaming_response = chat_engine.stream_chat("Tell me a joke.")
for token in streaming_response.response_gen:
    print(token, end="")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
